<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../../data/test_sessions (1).csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [4]:
train_df.shape

(253561, 21)

In [49]:
# train_df[times] = train_df[times].fillna(0).apply(pd.to_datetime)#крч нужно в тип время год перевести
#  test_df[times] = test_df[times].fillna(0).astype('datetype')



In [4]:
train_df[sites].head()

NameError: name 'sites' is not defined

В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [6]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)] 

train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')


# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [7]:
train_df.shape, test_df.shape, full_df.shape


NameError: name 'full_df' is not defined

Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [8]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]


Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [14]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [15]:
from scipy.sparse import csr_matrix

In [35]:
csr_matrix?

In [16]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]
full_sites_sparse.shape #336к сессий и 48к уникальных сайтов из них только 10 любимые Элис


(336358, 48371)

In [17]:
X_test_sparse = full_sites_sparse[idx_split:]
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse.shape, X_train_sparse.shape, y_train.shape


((82797, 48371), (253561, 48371), (253561,))

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [18]:
def get_auc_lr_valid(X, y, C=5.99484250e-02, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    
    # Ваш код здесь
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    logit = LogisticRegression(C = C ,n_jobs=-1, random_state=seed)
    logit.fit(X_train,y_train)
    valid_pred = logit.predict_proba(X_valid)[:,1]
    return roc_auc_score(y_valid, valid_pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [19]:
get_auc_lr_valid(X_train_sparse, y_train)

/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.9202564188491539

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [20]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [21]:
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse,y_train)
logit.predict_proba(X_test_sparse[:15, :]) #выведет два столбца, слева то что это не Элис, а справа наоборот
                                             # и еще [:15, :] означает что первые 15 строчек и все столбцы
test_pred = logit.predict_proba(X_test_sparse)[:,1] #[:,1] означает что выведет только пробабилити что это Элис
test_pred

/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


array([2.20959116e-03, 4.80930622e-09, 1.87275185e-08, ...,
       8.62756718e-03, 4.31261454e-04, 2.02775884e-05])

In [22]:
test_pred[:5], test_df.index[:5]
write_to_submission_file(test_pred, 'Alice.csv')
!head Alice.csv


session_id,target
1,0.002209591162799229
2,4.809306217921277e-09
3,1.872751847952394e-08
4,2.3541826080089027e-08
5,3.1309919441096034e-05
6,0.00021846560139161366
7,0.0005479178059957567
8,0.0001322781845147674
9,0.000795190489294533


Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [23]:
new_trfeat = pd.DataFrame(index = train_df.index)
new_tsfeat = pd.DataFrame(index = test_df.index)
new_trfeat['YearMonth'] = train_df['time1'].apply(lambda ts: 100*ts.year + ts.month)
new_tsfeat['YearMonth'] = test_df['time1'].apply(lambda ts: 100*ts.year + ts.month)


In [24]:
new_trfeat.head()

,YearMonth
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

<253561x48371 sparse matrix of type '<class 'numpy.int64'>'
	with 1429676 stored elements in Compressed Sparse Row format>

In [25]:
# Ваш код здесь
scaler = StandardScaler()
scaler.fit(new_trfeat['YearMonth'].values.reshape(-1, 1))
new_trfeat['YearMonthScaled'] = scaler.transform(new_trfeat['YearMonth'].values.reshape(-1, 1))
new_tsfeat['YearMonthScaled'] = scaler.transform(new_tsfeat['YearMonth'].values.reshape(-1, 1))
#кароче тут мы делаем что наш 'YearMonth' имеет слишком большое значение 201301(200к), в плане оптимизации наш код будет херовым,
#изза этого мы и делаем трансформ, YearMonthScale = median-mean помоему
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, (new_trfeat['YearMonthScaled'].values.reshape(-1,1))]))#тут добавили новый признак к старым данным
X_train_sparse_new.shape, X_train_sparse.shape
X_test_sparse_new = csr_matrix(hstack([X_test_sparse, (new_tsfeat['YearMonthScaled'].values.reshape(-1,1))]))#тут добавили новый признак к старым данным
get_auc_lr_valid(X_train_sparse_new, y_train)




/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.920045234757007

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [26]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


In [27]:
start_hour_train = pd.DataFrame(index = train_df.index)
start_hour_test = pd.DataFrame(index = test_df.index)
start_hour_train['hour'] = train_df['time1'].apply(lambda hr: hr.hour)
start_hour_test['hour'] = test_df['time1'].apply(lambda hr: hr.hour)
start_hour_train.head()

scaler = StandardScaler()
scaler.fit(start_hour_train['hour'].values.reshape(-1, 1))
start_hour_train['hourScaled'] = scaler.transform(start_hour_train['hour'].values.reshape(-1, 1))
start_hour_test['hourScaled'] = scaler.transform(start_hour_test['hour'].values.reshape(-1, 1))
X_train_sparse_new_hour = csr_matrix(hstack([X_train_sparse_new, (start_hour_train['hourScaled'].values.reshape(-1,1))]))#тут добавили новый признак к старым данным
X_test_sparse_new_hour = csr_matrix(hstack([X_test_sparse_new, (start_hour_test['hourScaled'].values.reshape(-1,1))]))#тут добавили новый признак к старым данным

X_train_sparse_new_hour.shape, X_test_sparse_new_hour.shape




/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


((253561, 48373), (82797, 48373))

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [80]:
C = 1.0  >>>0.9579181354452346
C = 1.00000000e-03 >>>0.8229544428060946
C = 2.78255940e-03 >>>0.9030958621805671
C = 7.74263683e-03 >>>0.9426473669464742
C = 2.15443469e-02 >>>0.957613511072145
C = 5.99484250e-02 >>>0.9621866505085928
C = 1.66810054e-01 >>>0.9621529878548768
C = 4.64158883e-01 >>>0.960031787809953
C = 1.29154967e+00 >>>0.9570549827369458
C = 1.00000000e+01 >>>0.9459435898457738
C = 3.59381366e+00 >>>0.9519196920788002

SyntaxError: invalid syntax (<ipython-input-80-31ee25a5157c>, line 1)

In [28]:

get_auc_lr_valid(X_train_sparse_new_hour, y_train)

/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.9621866505085928

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [29]:
np.logspace(-3,1,10)

array([1.00000000e-03, 2.78255940e-03, 7.74263683e-03, 2.15443469e-02,
       5.99484250e-02, 1.66810054e-01, 4.64158883e-01, 1.29154967e+00,
       3.59381366e+00, 1.00000000e+01])

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [30]:
morning_train = pd.DataFrame(index = train_df.index)
morning_test = pd.DataFrame(index = test_df.index)
morning_train['hour'] = train_df['time1'].apply(lambda hr: hr.hour)
morning_test['hour'] = test_df['time1'].apply(lambda hr: hr.hour)

ind = morning_train['hour'].values.shape[0]
for i in range(ind):
    if (morning_train['hour'].values[i] <= 11):
        morning_train['hour'].values[i] = 1
    else:
        morning_train['hour'].values[i] = 0
morning_train['hour']

ind2 = morning_test['hour'].values.shape[0]
for i in range(ind2):
    if (morning_test['hour'].values[i] <= 11):
        morning_test['hour'].values[i] = 1
    else:
        morning_test['hour'].values[i] = 0
morning_test['hour']

    
  


scaler = StandardScaler()
scaler.fit(morning_train['hour'].values.reshape(-1, 1))
morning_train['hour'] = scaler.transform(morning_train['hour'].values.reshape(-1, 1))
morning_test['hour'] = scaler.transform(morning_test['hour'].values.reshape(-1, 1))
X_train_sparse_new_hour_morning = csr_matrix(hstack([X_train_sparse_new_hour, (morning_train['hour'].values.reshape(-1,1))]))#тут добавили новый признак к старым данным
X_test_sparse_new_hour_morning = csr_matrix(hstack([X_test_sparse_new_hour, (morning_test['hour'].values.reshape(-1,1))]))#тут добавили новый признак к старым данным

X_train_sparse_new_hour.shape, X_test_sparse_new_hour.shape, X_test_sparse_new_hour_morning.shape, X_train_sparse_new_hour_morning.shape
morning_test

/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,hour
session_id,
1,1.039061
2,1.039061
3,-0.962408
4,1.039061
5,-0.962408
6,1.039061
7,-0.962408
8,-0.962408
9,1.039061


In [31]:
get_auc_lr_valid(X_train_sparse_new_hour_morning, y_train)

/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.9606967384360482

In [32]:
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse_new_hour_morning,y_train)
logit.predict_proba(X_test_sparse_new_hour_morning[:15, :])
test_pred = logit.predict_proba(X_test_sparse_new_hour_morning)[:,1]

/home/norton/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
